In [24]:
import numbers
#'pip install git+git://github.com/clintval/gender_predictor.git'
from gender_predictor import GenderPredictor
import itertools

In [25]:
from sherlock import helpers
from sherlock.features.preprocessing import extract_features, convert_string_lists_to_lists, prepare_feature_extraction
from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock

# Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()`awill download 3.6GB of data into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [26]:
helpers.download_data()
prepare_feature_extraction()

Data was downloaded.
Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


# Read input dataset

In [31]:
tmp_samples = pd.read_csv('adult.csv')

In [23]:
tmp_samples

ImportError: cannot import name 'is_url'

       age         workclass  fnlwgt     education  education.num  \
0       90                 ?   77053       HS-grad              9   
1       82           Private  132870       HS-grad              9   
2       66                 ?  186061  Some-college             10   
3       54           Private  140359       7th-8th              4   
4       41           Private  264663  Some-college             10   
5       34           Private  216864       HS-grad              9   
6       38           Private  150601          10th              6   
7       74         State-gov   88638     Doctorate             16   
8       68       Federal-gov  422013       HS-grad              9   
9       41           Private   70037  Some-college             10   
10      45           Private  172274     Doctorate             16   
11      38  Self-emp-not-inc  164526   Prof-school             15   
12      52           Private  129177     Bachelors             13   
13      32           Private  1362

# Test whether the input dataset contains header

In [8]:
def identify_header(path, n=5, th=0.9):
    df1 = pd.read_csv(path, header='infer', nrows=n)
    df2 = pd.read_csv(path, header=None, nrows=n)
    sim = (df1.dtypes.values == df2.dtypes.values).mean()
    return 'infer' if sim < th else None

In [9]:
if identify_header('adult.csv', n=5, th=0.9) == 'infer':
    header = True
else:
    header = False

In [10]:
sensitive = ['age','area','nation','nationality','sex','gender','ethnicity','race']
candidate = []
if header:
    # Check the matched sensitive attributes
    for i in tmp_samples.columns:
        if i.lower() in sensitive:
            candidate.append(i)

# Generate Header

In [11]:
# Convert input dataset to the required form
index_range = range(len(tmp_samples.columns))
df_value = pd.DataFrame(columns = ['value'],index = index_range)
df_label = pd.DataFrame(columns = ['label'],index = index_range)

In [12]:
idx = 0
for i in tmp_samples.columns:
    unique_val = list(tmp_samples[i].unique())
    val_no_nan = [x for x in unique_val if str(x) != 'nan']
    if len(val_no_nan) == 0:
        # No value in any cell of this attribute
        val_no_nan = [i]
    df_value.at[idx,'value'] = str(val_no_nan)
    df_label.at[idx,'label'] = i
    idx += 1

AbstractMethodError: This method must be defined in the concrete class type

In [13]:
# Load Sherlock architecture and weights from files
file = open('../models/sherlock_model.json', 'r')
sherlock_file = file.read()
sherlock = tf.keras.models.model_from_json(sherlock_file)
file.close()

sherlock.load_weights('../models/sherlock_weights.h5')
sherlock.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['categorical_accuracy'])

W1201 01:28:51.364897 4383374848 deprecation.py:506] From /Users/max/opt/anaconda3/envs/cs576/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1201 01:28:51.367347 4383374848 deprecation.py:506] From /Users/max/opt/anaconda3/envs/cs576/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1201 01:28:51.373263 4383374848 deprecation.py:506] From /Users/max/opt/anaconda3/envs/cs576/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init_

In [128]:
test_samples_converted, y_test = convert_string_lists_to_lists(df_value, df_label,'value','label')

100%|██████████| 15/15 [00:00<00:00, 337.65it/s]


In [129]:
X_test = extract_features(test_samples_converted)

Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


In [130]:
predicted_labels = predict_sherlock(X_test, nn_id='sherlock')

In [131]:
# If there is no header, assign the generated header to the dataset
if not header:
    tmp_samples.columns = predicted_labels

In [132]:
# Filter out sensitive attributes
for i in zip(tmp_samples.columns,predicted_labels):
    if i not in candidate:
        if i in ['country','sex','age']:
            candidate.append(i)

# Test whether the attribute is categorical

In [133]:
def summary_age(x):
    age = {'<16':0,'16-24':0,'25-34':0,
          '35-44':0,'45-54':0,'55-64':0,'>64':0}
    for i in x:
        if i < 16:
            age['<16'] += 1
        elif i <= 24:
            age['16-24'] += 1
        elif i <= 34:
            age['25-34'] += 1
        elif i <= 44:
            age['35-44'] += 1
        elif i <= 54:
            age['45-54'] += 1
        elif i <= 64:
            age['55-64'] += 1
        else:
            age['>64'] += 1
    return age

In [134]:
data_summary = []
for i in tmp_samples.columns:
    if i in candidate:
        # It must be categorical
        if i.lower() in ['age']:
            summary = summary_age(tmp_samples[i])
        else:
            summary = tmp_samples[i].value_counts()
    else:
        # check whether it is categorical or not
        if len(tmp_samples[i].unique()) / tmp_samples.shape[0] < 0.2:
            if not(isinstance(tmp_samples[i].dtype, int) or np.issubdtype(tmp_samples[i].dtype, np.int) or 
                isinstance(tmp_samples[i].dtype, float) or np.issubdtype(tmp_samples[i].dtype, np.float)):
                # we determine it is categorical
                summary = tmp_samples[i].value_counts()
    data_summary.append(summary)

/Users/max/opt/anaconda3/envs/cs576/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sys.path[0] == '':
/Users/max/opt/anaconda3/envs/cs576/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  del sys.path[0]


# Further, check what we can extend

##### First, predict the gender based on first name, if there is no gender attribute but with name attribute

In [135]:
gp = GenderPredictor()
gp.train_and_test()

import complete
32,031 male names
56,347 female names
classifier accuracy: 96.81%


In [136]:
get_gender = get_name = False
name = None
for i in zip(tmp_samples.columns, predicted_labels):
    if (i[0].lower() in ['name','first_name','first','last','last_name','first name','last name']) or (i[1].lower() in ['name','first_name','first','last','last_name','first name','last name']):
        get_name =True
        name = i[0]
    if i[0].lower() in ['sex','gender'] or i[1].lower() in ['sex']:
        get_gender = True
if get_name and (not get_gender):
    male = female = 0
    for i in tmp_samples[i]:
        if gp.classify(i) == 'M':
            male += 1
        else:
            female += 1
    data_summary.append({'Male':male,'Female':female})

# Try to extend combined sensitive attributes information

In [141]:
candidate

['age', 'race', 'sex']

In [160]:
for L in range(2, len(candidate)+1):
    for subset in itertools.combinations(candidate, L):
        data_summary.append(tmp_samples.groupby(list(subset)).size())